In [31]:
import pandas as pd
import numpy as np
from geopy import geocoders, Nominatim
from geopy.distance import great_circle as GC  

import json
import requests
import networkx as nx

import random
import itertools 

import plotly.graph_objects as go
import plotly.express as px

import folium
import geopandas as gpd
import earthpy as et

## 1) Import Coordinates from European Capitals:

In [32]:
# Define list of cities:
cities = [
    'Berlin',
    'Munich',
    'Hamburg',
    'Bielefeld',
    'Münster',
    'Stuttgart',
    'Karlsruhe',
    'Duesseldorf',
    # 'Cologne',
    'Osnabrueck',
    'Frankfurt',
    'Goettingen',
    'Siegen',
    'Kassel',
    'Hannover',
    'Bremen',
    'Dresden'
]

# Sort list in alphabetical order:
cities.sort()

# Get coordinates and create list of cities including name and coordinates:
geolocator = Nominatim(user_agent="TSP_GermanCities")

city_and_coords = []
for city in cities:
    
    loc = geolocator.geocode(city)
    
    city_and_coords.append({
        'city': city,
        'coord': (loc.latitude,loc.longitude)
    })
    
print(city_and_coords)

[{'city': 'Berlin', 'coord': (52.5186925, 13.3996024)}, {'city': 'Bielefeld', 'coord': (52.0191005, 8.531007)}, {'city': 'Bremen', 'coord': (53.0758196, 8.8071646)}, {'city': 'Dresden', 'coord': (51.0493286, 13.7381437)}, {'city': 'Duesseldorf', 'coord': (51.19077325, 6.793516713552313)}, {'city': 'Frankfurt', 'coord': (50.1106444, 8.6820917)}, {'city': 'Goettingen', 'coord': (51.50032665, 9.950655394459552)}, {'city': 'Hamburg', 'coord': (53.550341, 10.000654)}, {'city': 'Hannover', 'coord': (52.3744779, 9.7385532)}, {'city': 'Karlsruhe', 'coord': (49.0068705, 8.4034195)}, {'city': 'Kassel', 'coord': (51.3154546, 9.4924096)}, {'city': 'Munich', 'coord': (48.1371079, 11.5753822)}, {'city': 'Münster', 'coord': (51.9625101, 7.6251879)}, {'city': 'Osnabrueck', 'coord': (52.2733042, 8.0455442)}, {'city': 'Siegen', 'coord': (50.8749804, 8.0227233)}, {'city': 'Stuttgart', 'coord': (48.7784485, 9.1800132)}]


## 2) Compute Distance Matrix

In [33]:
# create variable coords as a list containing only the coordinates as tuples
coords = []
for city in city_and_coords:
    coords.append(city['coord'])

def get_distance(point1: tuple, point2: tuple) -> int:

    # air distance
    dist = GC(point1, point2).km

    return round(dist,4)


def compute_dist_matrix(coords: list, city_names:list, verbose=False, format='DataFrame') -> pd.DataFrame:
    # Create empty np-array:
    dist_array = np.empty((len(coords),len(coords)))
    
    # Compute distances:
    for i in range(0,len(coords)):
        for j in range(0,len(coords)):
            if i < j:
                dist = get_distance(coords[i],coords[j])
                
                if verbose:
                    print('Distance between: ' + city_names[i] + ' and ' + city_names[j] + ': ' + str(dist) +'km')

                dist_array[i][j] = dist
                dist_array[j][i] = dist

            elif i == j:
                dist_array[i][i] = 0
                
    # Create pandas distance matrix:
    dist_array_df = pd.DataFrame(data=dist_array, index=city_names, columns=city_names)
    
    if format == 'NumpyArray':
        return dist_array
    else:
        return dist_array_df

In [34]:
coords

[(52.5186925, 13.3996024),
 (52.0191005, 8.531007),
 (53.0758196, 8.8071646),
 (51.0493286, 13.7381437),
 (51.19077325, 6.793516713552313),
 (50.1106444, 8.6820917),
 (51.50032665, 9.950655394459552),
 (53.550341, 10.000654),
 (52.3744779, 9.7385532),
 (49.0068705, 8.4034195),
 (51.3154546, 9.4924096),
 (48.1371079, 11.5753822),
 (51.9625101, 7.6251879),
 (52.2733042, 8.0455442),
 (50.8749804, 8.0227233),
 (48.7784485, 9.1800132)]

In [35]:
# create distance matrix with two different formats: np.Array and pd.DataFrame
dist_matrix_np = compute_dist_matrix(coords,cities,verbose=False, format='NumpyArray')
dist_matrix_df = compute_dist_matrix(coords,cities,verbose=False, format='DataFrame')

# create dictionary to convert from index to city and vice versa:
index_to_city = {}
city_to_index = {}
for i,city in zip(range(0,len(cities)),cities):
    index_to_city[i] = city
    city_to_index[city] = i
    

# print dist matrix:
dist_matrix_df

,Berlin,Bielefeld,Bremen,Dresden,Duesseldorf,Frankfurt,Goettingen,Hamburg,Hannover,Karlsruhe,Kassel,Munich,Münster,Osnabrueck,Siegen,Stuttgart
Berlin,0.0000,335.8487,314.8566,165.0368,476.9274,423.1692,261.7780,254.5455,248.6129,525.0843,299.4648,504.0817,397.9020,364.2186,413.0609,511.1601
Bielefeld,335.8487,0.0000,118.9762,375.8700,151.2603,212.4733,113.4624,196.8640,91.2963,335.0665,102.5557,483.1370,62.3418,43.5460,132.0060,363.2648
Bremen,314.8566,118.9762,0.0000,405.3340,250.6272,329.8259,191.6699,95.2370,100.0783,453.3248,201.2363,582.7572,147.3759,102.9542,250.5474,478.5587
Dresden,165.0368,375.8700,405.3340,0.0000,484.7826,371.8607,268.1632,376.6347,312.4249,443.4585,297.3691,359.3530,434.9571,415.4325,400.6443,412.5242
Duesseldorf,476.9274,151.2603,250.6272,484.7826,0.0000,179.3050,221.9457,340.8797,241.5640,268.6039,188.3294,483.2822,103.2794,148.0627,92.8500,317.8697
Frankfurt,423.1692,212.4733,329.8259,371.8607,179.3050,0.0000,178.3879,393.0447,262.2419,124.3690,145.6095,304.0686,218.7715,244.5315,96.9473,152.4439
Goettingen,261.7780,113.4624,191.6699,268.1632,221.9457,178.3879,0.0000,227.9766,98.2827,298.2673,37.8524,391.6918,168.1866,156.4636,151.2849,307.5973
Hamburg,254.5455,196.8640,95.2370,376.6347,340.8797,393.0447,227.9766,0.0000,131.9231,517.2468,250.8839,611.9540,238.1501,193.2511,326.5553,533.6776
Hannover,248.6129,91.2963,100.0783,312.4249,241.5640,262.2419,98.2827,131.9231,0.0000,386.0734,118.9658,488.8929,151.2305,115.6064,204.5120,401.7978
Karlsruhe,525.0843,335.0665,453.3248,443.4585,268.6039,124.3690,298.2673,517.2468,386.0734,0.0000,268.1601,252.5986,333.2275,364.0858,209.5027,62.1970


### Add helper functions:

In [36]:
def get_length_of_route(dist_matrix:np.array, route: list) -> int:
    length = 0
    
    # iterate over cities in route (start with 1 because index for origin is i-1):
    for i in range(1,len(route)):
        origin = route[i-1]
        dest = route[i]

        length += dist_matrix[origin][dest]
        
    # add last trip from starting_point to last destination:
    length += dist_matrix[route[0]][route[len(route)-1]]
    
    return length
        

In [37]:
def swap_position_of_two_elements_in_list(list, pos1, pos2):
    
    copy = list.copy()
    
    copy[pos1], copy[pos2] = copy[pos2], copy[pos1]
    
    return copy

In [38]:
def adjust_start_idx_to_front(route, start_idx):

    # find index of start city:
    idx = route.index(start_idx)

    # get all elements from desired index until the end of the list
    first_part = route[idx:len(route)]
    # get all elements from the beginning of the list until one element before the desired index
    second_part = route[0:idx]

    # glue both parts together
    result = first_part+second_part

    return result

In [39]:
def map_city_names_to_route(route_indices:list, names_dict:dict, type='list_with_names') -> list:
    
    route_with_names = []
    
    if type == 'list_with_names':
        for idx in route_indices:
            route_with_names.append(names_dict[idx])

        return route_with_names
    
    elif type == 'list_with_names_and_idx':
        for idx in route_indices:
            route_with_names.append({
                'name': names_dict[idx],
                'idx': idx
            })

        return route_with_names
    
    else:
        raise ValueError('unknown return type passed')
    
        
    

## 3) Traveling Salesman Algorithms:

### Randomized Approach:
Idea: Create random routes and choose the best route out of a predefined number of attempts.

In [40]:
def random_algorithm(dist_matrix:np.array, start_idx=-1, n_iter=100, verbose=True) -> list:
    
    # initialize best route and length
    best_route = []
    best_length = float('inf')
    
    # get number of cities:
    n_cities = len(cities)
    
    # set random starting point if nothing was passed:
    if start_idx == -1:
        start_idx = random.randint(0,n_cities-1)
        
    for i in range(0,n_iter):
        
        # random route
        route = list(np.random.permutation(n_cities))
        
        # get index of starting point city:
        idx = route.index(start_idx)
        
        # swap starting point city to beginning:
        route = swap_position_of_two_elements_in_list(route, 0, idx)
        
        # compute distance:
        dist = get_length_of_route(dist_matrix, route)
        
        # keep if better than previous attempts:
        if dist < best_length:
            best_route = route
            best_length = dist
            
            if verbose:
                print('New best_length: ' + str(dist))
            
    return best_route

### Greedy approach
Idea: Always proceed to the closest next city

In [41]:
def greedy_algorithm(dist_matrix:np.array, start_idx=-1) -> list:
    
    # initialize route
    route = []
    # get number of cities:
    n_cities = len(cities)
    # initialize remaining cities:
    cities_remaining = list(range(0,n_cities))
    
    
    # set random starting point if nothing was passed:
    if start_idx == -1:
        start_idx = random.randint(0,n_cities-1)
    
    # add starting point to route:
    route.append(start_idx)
    # remove starting point from remaining cities:
    cities_remaining.remove(start_idx)
    

    # add cities to route until every city is added:
    while cities_remaining:
        
        # get city that was added last:
        idx_added_prev = route[-1]

        # initialize min_distance
        idx_closest_city = cities_remaining[0]
        min_dist = dist_matrix_np[idx_closest_city][idx_added_prev]
                
        # find closest city to cities in remaining cities:
        for city_idx in cities_remaining:
            dist = dist_matrix_np[city_idx][idx_added_prev]
            
            if dist <= min_dist:
                min_dist = dist
                idx_closest_city = city_idx
    
        # add city to route and remove it from remaining cities:
        route.append(idx_closest_city)
        cities_remaining.remove(idx_closest_city)
    
    return route
    

### 2-opt swap: 
Idea: Start with random route. Then optimize intermediate result by exchanging order of two cities if the overall result is improved.

In [42]:
def opt2_algorithm(dist_matrix:np.array, n_iter=1000, max_rounds=10**6, verbose=False, start_idx=-1) -> list:
    
    # Get number of cities:
    n_cities = len(cities)
    
    # initialize best attempts:
    best_route = list(np.random.permutation(n_cities))

    # initialize best length:
    best_length = get_length_of_route(dist_matrix, best_route)

    
    # n_iter attempts:
    for t in range(0,n_iter):
    
        # initialize route with random permutation:
        route = list(np.random.permutation(n_cities))

        # swap start_idx city with other city:
        if start_idx != -1:
            # get position of start city in list:
            pos_start_city = best_route.index(start_idx)

        # initialize route_length:
        route_length = get_length_of_route(dist_matrix, route)

        # variable that keeps track if swapping order of two cities led to an improvement:
        improvement = True
        # variable that counts number of rounds:
        swap_round = 0

        while improvement and swap_round < max_rounds:
            # at each round start by setting
            improvement = False

            # iterate over cities:
            for i in range(0,n_cities):
                for j in range(i,n_cities):

                    # swap two elements:
                    potential_new_route = swap_position_of_two_elements_in_list(route,i,j)

                    # compute new distance:
                    potential_new_distance = get_length_of_route(dist_matrix, potential_new_route)

                    # check if improvement:
                    if potential_new_distance < route_length:

                        # print('Improvement by ' + str(route_length - potential_new_distance))
                        # print('New distance: ' + str(potential_new_distance))

                        # set boolean flag to true:
                        improvement = True

                        # update route and route_length:
                        route = potential_new_route
                        route_length = potential_new_distance

            # count number of rounds:
            swap_round += 1
                    
        if route_length < best_length:
            best_route = route
            best_length = route_length
        
        # Print progress if verbose is set to true
        if verbose:
            perc_completed = (t+1)/n_iter*100
            
            if perc_completed % 10 == 0:
                print(str(perc_completed)+'% completed.')

    # swap start index to front:
    best_route = adjust_start_idx_to_front(best_route, start_idx)

    return best_route


In [43]:
def opt2_swap(dist_matrix:np.array, n_iter=1000, max_rounds=10**6) -> list:

    # Get number of cities:
    n_cities = len(cities)

    # initialize best attempts:
    best_route = list(np.random.permutation(n_cities))

    # initialize best length:
    best_length = get_length_of_route(dist_matrix, best_route)


    # n_iter attempts:
    for t in range(0,n_iter):

        # initialize route with random permutation:
        route = list(np.random.permutation(n_cities))

        # swap start_idx city with other city:
        if start_idx != -1:
            # get position of start city in list:
            pos_start_city = best_route.index(start_idx)

        # initialize route_length:
        route_length = get_length_of_route(dist_matrix, route)

        # variable that keeps track if swapping order of two cities led to an improvement:
        improvement = True
        # variable that counts number of rounds:
        swap_round = 0

        while improvement and swap_round < max_rounds:
            # at each round start by setting
            improvement = False

            # iterate over cities:
            for i in range(0,n_cities):
                for j in range(i,n_cities):

                    # swap two elements:
                    potential_new_route = swap_position_of_two_elements_in_list(route,i,j)

                    # compute new distance:
                    potential_new_distance = get_length_of_route(dist_matrix, potential_new_route)

                    # check if improvement:
                    if potential_new_distance < route_length:

                        # print('Improvement by ' + str(route_length - potential_new_distance))
                        # print('New distance: ' + str(potential_new_distance))

                        # set boolean flag to true:
                        improvement = True

                        # update route and route_length:
                        route = potential_new_route
                        route_length = potential_new_distance

            # count number of rounds:
            swap_round += 1

        if route_length < best_length:
            best_route = route
            best_length = route_length

        # Print progress if verbose is set to true
        if verbose:
            perc_completed = (t+1)/n_iter*100

            if perc_completed % 10 == 0:
                print(str(perc_completed)+'% completed.')

    # swap start index to front:
    best_route = adjust_start_idx_to_front(best_route, start_idx)

    return best_route


In [44]:
# Set starting city:
starting_city = 'Münster'
# Get index of starting city:
start_city_idx = dist_matrix_df.columns.get_loc(starting_city)

In [45]:
random_route = random_algorithm(dist_matrix_np, n_iter=1, verbose=True, start_idx=start_city_idx)

get_length_of_route(dist_matrix_np, random_route)

New best_length: 4647.835999999999


4647.835999999999

In [46]:
greedy_route = greedy_algorithm(dist_matrix_np, start_idx=start_city_idx)

get_length_of_route(dist_matrix_np, greedy_route)

2100.3675000000003

In [47]:
opt2_route = opt2_algorithm(dist_matrix_np, verbose=False, start_idx=start_city_idx)

get_length_of_route(dist_matrix_np, opt2_route)

1971.5324999999998

In [62]:
route = greedy_route

## 4) Visualization

In [63]:
# Create DataFrame in desired format:
# city, lat, lon
route_dict_inside_list = []

for idx in route:
    route_dict_inside_list.append({
        'city':city_and_coords[idx]['city'],
        'lon':city_and_coords[idx]['coord'][0],
        'lat':city_and_coords[idx]['coord'][1],
    })
    
path_df = pd.DataFrame(route_dict_inside_list)

path_df

,city,lon,lat
0,Münster,51.962510,7.625188
1,Osnabrueck,52.273304,8.045544
2,Bielefeld,52.019101,8.531007
3,Hannover,52.374478,9.738553
4,Goettingen,51.500327,9.950655
5,Kassel,51.315455,9.492410
6,Siegen,50.874980,8.022723
7,Duesseldorf,51.190773,6.793517
8,Frankfurt,50.110644,8.682092
9,Karlsruhe,49.006870,8.403420


In [64]:
# Compute center point:
center_point_coords = path_df.iloc[:,1:3].mean().tolist()

# Start city:
start_point = route_dict_inside_list[0]

# Get routes coordinates as a list
route_list = []
for city in route_dict_inside_list:
    route_list.append((city['lon'], city['lat']))

# Add last route from end_point to starting point:
route_list.append((start_point['lon'], start_point['lat']))

### Helper functions for plotting:

In [65]:
# Function for creating icons with numbers:
def create_custom_icon(color='blue', number=''):
    icon = folium.DivIcon(
        icon_size=(150,36),
        icon_anchor=(14,40),
        #             html='<div style="font-size: 18pt; align:center, color : black">' + '{:02d}'.format(num+1) + '</div>',
        html="""<span class="fa-stack " style="font-size: 12pt" >>
                    <!-- The icon that will wrap the number -->
                    <span class="fa fa-circle-o fa-stack-2x" style="color : {:s}"></span>
                    <!-- a strong element with the custom content, in this case a number -->
                    <strong class="fa-stack-1x">
                         {:02d}
                    </strong>
                </span>""".format(color,number)
    )

    return icon


In [66]:
# create map object:
map_osm = folium.Map(location=center_point_coords, zoom_start=6, tiles='Open Street Map')

# add starting marker:
folium.Marker(location=(start_point['lon'], start_point['lat']), popup=start_point['city'], icon=folium.Icon(color='gray', icon='home'),  tooltip='Click here!', draggable=False).add_to(map_osm)

# add markers for other cities:
for i, city in enumerate(route_dict_inside_list[1:]):

    # create icon:
    icon = folium.Icon(color='darkblue', icon='car', prefix='fa')
    ugly_icon = create_custom_icon(color='blue', number=i+1)


    # folium.Marker(location=(city['lon'], city['lat']), popup=city['city'], icon=ugly_icon,  tooltip=city['city'], draggable=False).add_to(map_osm)
    folium.Marker(location=(city['lon'], city['lat']), popup=city['city'], icon=folium.Icon(color='darkblue', icon='home'),  tooltip=city['city'], draggable=False).add_to(map_osm)

# plot routes:
folium.PolyLine(route_list,
                color="black",
                weight=3).add_to(map_osm)

map_osm

In [67]:
route_list

[(51.9625101, 7.6251879),
 (52.2733042, 8.0455442),
 (52.0191005, 8.531007),
 (52.3744779, 9.7385532),
 (51.50032665, 9.950655394459552),
 (51.3154546, 9.4924096),
 (50.8749804, 8.0227233),
 (51.19077325, 6.793516713552313),
 (50.1106444, 8.6820917),
 (49.0068705, 8.4034195),
 (48.7784485, 9.1800132),
 (48.1371079, 11.5753822),
 (51.0493286, 13.7381437),
 (52.5186925, 13.3996024),
 (53.550341, 10.000654),
 (53.0758196, 8.8071646),
 (51.9625101, 7.6251879)]